#Exercise 3

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 18.2 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import pandas as pd

##Ans 1 :

$\text{min }\sum_{i=1}^{6}|P_i-\widetilde P_i|$

$\widetilde P_i = b_0 + b_1L_i+b_2E_i$

$b_0\geq0,b_0\in\mathbb{R} \\
b_1\in\mathbb{R} \\
b_2\in\mathbb{R}$

This problem is not linear since we have an absolute function as the objective function.



In [4]:
data_csvfile = pd.read_csv('lab6_ex3.csv')

In [5]:
print(data_csvfile)

   Warehouse      SP  Lot Size  Elevation
0          1  175000      1800        250
1          2  150000      2800        400
2          3  125000      3500        450
3          4   90000       800        100
4          5   80000       700        200
5          6  120000       500         50


In [6]:
model = ConcreteModel()

In [7]:
data_csvfile.index

RangeIndex(start=0, stop=6, step=1)

In [8]:
N = len(data_csvfile.index)

In [9]:
print(N)

6


In [10]:
col_indices = range(N)

In [11]:
model.u = Var(col_indices, domain = NonNegativeReals)

In [12]:
model.b0 = Var(domain = NonNegativeReals)
model.b1 = Var()
model.b2 = Var()

In [13]:
model.objective = Objective(expr = summation(model.u), sense = minimize)

In [14]:
model.constraints = ConstraintList()

In [15]:
for i in col_indices:
  model.constraints.add(data_csvfile['SP'][i] - (model.b0 + model.b1*data_csvfile['Lot Size'][i] + model.b2*data_csvfile['Elevation'][i]) <= model.u[i])
  model.constraints.add(-data_csvfile['SP'][i] + (model.b0 + model.b1*data_csvfile['Lot Size'][i] + model.b2*data_csvfile['Elevation'][i]) <= model.u[i])

In [16]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

4 Var Declarations
    b0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    b1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    b2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    u : Size=6, Index=u_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegati

In [17]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [18]:
opt_cbc = SolverFactory('cbc')

In [19]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [30]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('u[',i,']', model.u[i].value)

print('\nb[0]=',model.b0.value)
print('b[1]=',model.b1.value)
print('b[2]=',model.b2.value)

print('\nConstraints')
model.constraints.display()


Objective =  125454.545

Decision Variables
u[ 0 ] 38818.182
u[ 1 ] 0.0
u[ 2 ] 54272.727
u[ 3 ] 32363.636
u[ 4 ] 0.0
u[ 5 ] 0.0

b[0]= 104181.82
b[1]= 67.272727
b[2]= -356.36364

Constraints
constraints : Size=12
    Key : Lower : Body                    : Upper
      1 :  None :  -0.0006000000576023012 :   0.0
      2 :  None :      -77636.36339999994 :   0.0
      3 :  None :  0.00039999998989515007 :   0.0
      4 :  None : -0.00039999998989515007 :   0.0
      5 :  None :     -108545.45350000005 :   0.0
      6 :  None :  -0.0004999999509891495 :   0.0
      7 :  None :     -64727.273600000015 :   0.0
      8 :  None :   0.0016000000177882612 :   0.0
      9 :  None :   -0.000900000028195791 :   0.0
     10 :  None :    0.000900000028195791 :   0.0
     11 :  None :   -0.001500000013038516 :   0.0
     12 :  None :    0.001500000013038516 :   0.0


##Ans 5 (Approach 1) :

Solver Status : Ok

Solver Termination Condition : Optimal

Optimal sum of residuals is : 125454.545

Values of new decision variables :

$u_1=38818.182\\
u_2=0.0\\
u_3=54272.727\\
u_4=32363.636\\
u_5=0.0\\
u_6=0.0$

Values of original decision variables :

$x_1=38818.182\\
x_2=0.0\\
x_3=-54272.727\\
x_4=-32363.636\\
x_5=0.0\\
x_6=0.0$

In [21]:
model_app_2 = ConcreteModel()

In [22]:
model_app_2.m = Var(col_indices,domain = NonNegativeReals)
model_app_2.n = Var(col_indices,domain = NonNegativeReals)

In [23]:
model_app_2.b0 = Var(domain = NonNegativeReals)
model_app_2.b1 = Var()
model_app_2.b2 = Var()

In [24]:
model_app_2.objective = Objective(expr = summation(model_app_2.m)+summation(model_app_2.n), sense = minimize)

In [25]:
model_app_2.constraints = ConstraintList()

In [26]:
for i in col_indices:
  model_app_2.constraints.add(data_csvfile['SP'][i] - (model_app_2.b0 + model_app_2.b1*data_csvfile['Lot Size'][i] + model_app_2.b2*data_csvfile['Elevation'][i]) <= model_app_2.m[i]+model_app_2.n[i])
  model_app_2.constraints.add(-data_csvfile['SP'][i] + (model_app_2.b0 + model_app_2.b1*data_csvfile['Lot Size'][i] + model_app_2.b2*data_csvfile['Elevation'][i]) <= model_app_2.m[i]+model_app_2.n[i])

In [27]:
model_app_2.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    m_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    n_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

5 Var Declarations
    b0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    b1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    b2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
 

In [28]:
result = opt_cbc.solve(model_app_2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [31]:
print('\nObjective = ', model_app_2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('m[',i,']', model_app_2.m[i].value)
  print('n[',i,']', model_app_2.n[i].value)

print('\nb[0]=',model_app_2.b0.value)
print('b[1]=',model_app_2.b1.value)
print('b[2]=',model_app_2.b2.value)

print('\nConstraints')
model_app_2.constraints.display()


Objective =  125454.545

Decision Variables
m[ 0 ] 38818.182
n[ 0 ] 0.0
m[ 1 ] 0.0
n[ 1 ] 0.0
m[ 2 ] 54272.727
n[ 2 ] 0.0
m[ 3 ] 0.0
n[ 3 ] 32363.636
m[ 4 ] 0.0
n[ 4 ] 0.0
m[ 5 ] 0.0
n[ 5 ] 0.0

b[0]= 104181.82
b[1]= 67.272727
b[2]= -356.36364

Constraints
constraints : Size=12
    Key : Lower : Body                    : Upper
      1 :  None :  -0.0006000000576023012 :   0.0
      2 :  None :      -77636.36339999994 :   0.0
      3 :  None :  0.00039999998989515007 :   0.0
      4 :  None : -0.00039999998989515007 :   0.0
      5 :  None :     -108545.45350000005 :   0.0
      6 :  None :  -0.0004999999509891495 :   0.0
      7 :  None :     -64727.273600000015 :   0.0
      8 :  None :   0.0016000000177882612 :   0.0
      9 :  None :   -0.000900000028195791 :   0.0
     10 :  None :    0.000900000028195791 :   0.0
     11 :  None :   -0.001500000013038516 :   0.0
     12 :  None :    0.001500000013038516 :   0.0


##Ans 5 (Approach 2) :

Solver Status : Ok

Solver Termination Condition : Optimal

Optimal sum of residuals is : 125454.545

Values of new decision variables :

$m_1=38818.182\\
n_1=0.0\\
m_2=0.0\\
n_2=0.0\\
m_3=54272.727\\
n_3=0.0\\
m_4=0.0\\
n_4=32363.636\\
m_5=0.0\\
n_5=0.0\\
m_6=0.0\\
n_6=0.0$

Values of original decision variables :

$x_1=38818.182\\
x_2=0.0\\
x_3=-54272.727\\
x_4=-32363.636\\
x_5=0.0\\
x_6=0.0$

We see that by both the approaches, we get the same optimal solution. In Approach 2, we also see that, $m_i*n_i=0$.

We see that the predicted value of 1st, 3rd and 4th warehouse is different by a significant value, whereas for 2nd, 5th and 6th warehouse is 0, by the calculated values of $b_0,b_1,b_2$